<a href="https://colab.research.google.com/github/Noam-shahar/AI-course/blob/main/HW5_Drone_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link to prompts: https://chat.openai.com/share/5478d433-fc4a-4efc-8b95-33d9a1563ac0

Clean slate:

In [1]:
import os

directory = '/content/sample_data'

# Check if the directory exists
if os.path.exists(directory):
    # Remove all files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")
else:
    print(f"Directory '{directory}' does not exist.")

Organizing:

In [2]:
import shutil

from google.colab import drive
drive.mount('/content/drive')


# Define the path to the new "Road2_drone_data" directory
copied_folder_path = '/content/sample_data/Road2_drone_data'

# Create the "Road2_drone_data" directory
os.makedirs(copied_folder_path)

# Create paths for the new folders within "Road2_drone_data"
true_drones_path = os.path.join(copied_folder_path, 'True_drones')
fake_ones_path = os.path.join(copied_folder_path, 'Fake_ones')
og_path = os.path.join(copied_folder_path, 'OG')

# Create the new folders
os.makedirs(true_drones_path)
os.makedirs(fake_ones_path)
os.makedirs(og_path)

# Define source and destination paths for the copy operation
source_path = '/content/drive/MyDrive/drone_data'
destination_path = os.path.join(og_path, os.path.basename(source_path))  # Destination path within OG

# Perform the copy operation
shutil.copytree(source_path, destination_path)

# Define the path to the "Testing" directory within "Road2_drone_data"
testing_folder_path = '/content/sample_data/Road2_drone_data/Testing'

# Create the "Testing" directory if it doesn't exist
os.makedirs(testing_folder_path, exist_ok=True)

# Create "True_drones" and "Fake_ones" folders within "Testing"
true_drones_testing_path = os.path.join(testing_folder_path, 'True_drones')
fake_ones_testing_path = os.path.join(testing_folder_path, 'Fake_ones')

# Create the new folders within "Testing"
os.makedirs(true_drones_testing_path, exist_ok=True)
os.makedirs(fake_ones_testing_path, exist_ok=True)

# Define paths
og_folder_path = '/content/sample_data/Road2_drone_data/OG'
drone_data_folder_path = '/content/sample_data/Road2_drone_data/OG/drone_data'

# Get the list of files in the drone_data folder
drone_data_files = os.listdir(drone_data_folder_path)

# Move files from drone_data folder to OG folder
for file_name in drone_data_files:
    source_file_path = os.path.join(drone_data_folder_path, file_name)
    destination_file_path = os.path.join(og_folder_path, file_name)
    shutil.move(source_file_path, destination_file_path)

# Define the path to the drone_data folder
drone_data_folder_path = '/content/sample_data/Road2_drone_data/OG/drone_data'

# Check if the directory is empty
if not os.listdir(drone_data_folder_path):
    # Directory is empty, delete it
    os.rmdir(drone_data_folder_path)
    print("Directory 'drone_data' deleted successfully.")
else:
    print("Directory 'drone_data' is not empty. Not deleted.")

Mounted at /content/drive
Directory 'drone_data' deleted successfully.


Process 1 - Fake drones (Testing):

In [3]:
import os
import cv2
import random

# Function to perform process 1: Randomly crop images to 28x28 and save in "Fake_ones" folder
def process_1(image_path, save_path):
    # Read the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale (black and white)

    if img is not None:  # Ensure image was read successfully
        # Get random coordinates for cropping
        h, w = img.shape[:2]
        start_h = random.randint(0, max(0, h - 28))
        start_w = random.randint(0, max(0, w - 28))

        # Crop the image to 28x28
        cropped_img = img[start_h:start_h+28, start_w:start_w+28]

        # Save the cropped image
        cv2.imwrite(save_path, cropped_img)

        # Save corresponding text file
        text_file_name = os.path.splitext(os.path.basename(save_path))[0] + '.txt'
        text_file_path = os.path.join(os.path.dirname(save_path), text_file_name)
        with open(text_file_path, 'w') as file:
            file.write("not a drone")
    else:
        print(f"Skipping {image_path} as it's not a valid image.")

# Define paths
og_folder_path = '/content/sample_data/Road2_drone_data/OG'
fake_ones_testing_path = '/content/sample_data/Road2_drone_data/Testing/Fake_ones'

# List image files in the OG folder
og_image_files = [file for file in os.listdir(og_folder_path) if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

# Process 5 random images from OG folder
if len(og_image_files) > 0:
    for i in range(min(5, len(og_image_files))):
        random_image_name = random.choice(og_image_files)
        random_image_path = os.path.join(og_folder_path, random_image_name)

        # Create corresponding paths for processed image and text file in Fake_ones
        processed_image_path = os.path.join(fake_ones_testing_path, random_image_name)

        # Perform Process 1: Random crop and save in "Fake_ones" folder
        process_1(random_image_path, processed_image_path)
else:
    print("No image files found in the 'OG' folder.")


Process 2 - Real drones (Testing):

In [4]:
# Import necessary libraries
import os                   # Operating System module for directory operations
import cv2                  # OpenCV for image processing
import random               # For random selection of images
from keras.preprocessing import image  # Keras for image preprocessing

# Function to extract bounding box coordinates from YOLOv8 format
def extract_bbox_from_yolov8(label_path, img_width, img_height):
    try:
        # Read lines from the label file
        with open(label_path, 'r') as file:
            data = file.readlines()

        # Check if label file has content
        if len(data) > 0:
            # Split the content by spaces and convert values to float
            bbox_values = list(map(float, data[0].split()))

            # Check if the number of values is sufficient for a bounding box
            if len(bbox_values) >= 4:
                # Extract the first 4 values as x_center, y_center, width, height
                x_center, y_center, width, height = bbox_values[1:]

                # Calculate bounding box coordinates (x, y, w, h)
                w, h = width * img_width, height * img_height  # Normalize to image dimensions
                x = (x_center * img_width) - (w / 2)
                y = (y_center * img_height) - (h / 2)

                return int(x), int(y), int(w), int(h)
            else:
                # Handle cases where insufficient values are found in label file
                print(f"Invalid format in label file '{label_path}': Expected at least 4 values for bbox, found {len(bbox_values)}")
                return None
        else:
            # Handle empty label files
            print(f"Empty label file '{label_path}'")
            return None
    except Exception as e:
        # Handle any exceptions during parsing
        print(f"Error parsing label file '{label_path}': {e}")
        return None

# Function to process images according to Process 2
def process_2(image_path, label_path, save_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale (black and white)

    if img is not None and img.size > 0:
        img_height, img_width = img.shape  # Updated line to get height and width (no channels for grayscale)

        bbox = extract_bbox_from_yolov8(label_path, img_width, img_height)

        if bbox is not None:
            x, y, w, h = bbox
            cropped_img = img[max(0, y):min(img_height, y + h), max(0, x):min(img_width, x + w)]

            # Check if the cropped image is valid and has the correct shape
            if cropped_img.size > 0:
                resized_img = cv2.resize(cropped_img, (28, 28))

                # Save the processed image
                cv2.imwrite(save_path, resized_img)

                # Create a text file (with the same name as the image) and write "drone" to it
                text_file_name = os.path.splitext(os.path.basename(save_path))[0] + '.txt'
                text_file_path = os.path.join(os.path.dirname(save_path), text_file_name)
                with open(text_file_path, 'w') as file:
                    file.write("drone")
            else:
                print(f"Skipping {image_path} due to invalid crop or shape.")
        else:
            print(f"Skipping {image_path} due to label parsing error.")
    else:
        print(f"Skipping {image_path} as it's not a valid image.")

# Define paths for the original images and the destination for processed images
og_folder_path = '/content/sample_data/Road2_drone_data/OG'
true_drones_testing_path = '/content/sample_data/Road2_drone_data/Testing/True_drones'

# Get a list of image files in the original folder
og_image_files = [file for file in os.listdir(og_folder_path) if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

# Process a maximum of 5 random images from the original folder
if len(og_image_files) > 0:
    for i in range(min(5, len(og_image_files))):
        random_image_name = random.choice(og_image_files)
        random_image_path = os.path.join(og_folder_path, random_image_name)

        label_file_name = os.path.splitext(random_image_name)[0] + '.txt'
        label_file_path = os.path.join(og_folder_path, label_file_name)

        processed_image_path = os.path.join(true_drones_testing_path, random_image_name)

        # Process the image according to Process 2
        process_2(random_image_path, label_file_path, processed_image_path)
else:
    print("No image files found in the 'OG' folder.")


Process 1 - Fake drones:

In [5]:
import os
import cv2
import random

# Function to perform process 1: Randomly crop images to 28x28 and save in "Fake_ones" folder
def process_1(image_path, save_path):
    # Read the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale (black and white)

    if img is not None:  # Ensure image was read successfully
        # Get random coordinates for cropping
        h, w = img.shape[:2]
        start_h = random.randint(0, max(0, h - 28))
        start_w = random.randint(0, max(0, w - 28))

        # Crop the image to 28x28
        cropped_img = img[start_h:start_h+28, start_w:start_w+28]

        # Save the cropped image
        cv2.imwrite(save_path, cropped_img)

        # Save corresponding text file
        text_file_name = os.path.splitext(os.path.basename(save_path))[0] + '.txt'
        text_file_path = os.path.join(os.path.dirname(save_path), text_file_name)
        with open(text_file_path, 'w') as file:
            file.write("not a drone")
    else:
        print(f"Skipping {image_path} as it's not a valid image.")

# Define paths
og_folder_path = '/content/sample_data/Road2_drone_data/OG'
fake_ones_path = '/content/sample_data/Road2_drone_data/Fake_ones'

# List image files in the OG folder
og_image_files = [file for file in os.listdir(og_folder_path) if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

# Process all images from OG folder using Process 1 (random cropping)
if len(og_image_files) > 0:
     for i in range(len(og_image_files)):
        random_image_name = og_image_files[i]  # Select images sequentially
        random_image_path = os.path.join(og_folder_path, random_image_name)

        # Create corresponding paths for processed image and text file in Fake_ones
        processed_image_path = os.path.join(fake_ones_path, random_image_name)

        # Perform Process 1: Random crop and save in "Fake_ones" folder
        process_1(random_image_path, processed_image_path)
else:
    print("No image files found in the 'OG' folder.")


Process 2 - Real drones:

In [6]:
# Import necessary libraries
import os                   # Operating System module for directory operations
import cv2                  # OpenCV for image processing
import random               # For random selection of images
from keras.preprocessing import image  # Keras for image preprocessing

# Function to extract bounding box coordinates from YOLOv8 format
def extract_bbox_from_yolov8(label_path, img_width, img_height):
    try:
        # Read lines from the label file
        with open(label_path, 'r') as file:
            data = file.readlines()

        # Check if label file has content
        if len(data) > 0:
            # Split the content by spaces and convert values to float
            bbox_values = list(map(float, data[0].split()))

            # Check if the number of values is sufficient for a bounding box
            if len(bbox_values) >= 4:
                # Extract the first 4 values as x_center, y_center, width, height
                x_center, y_center, width, height = bbox_values[1:]

                # Calculate bounding box coordinates (x, y, w, h)
                w, h = width * img_width, height * img_height  # Normalize to image dimensions
                x = (x_center * img_width) - (w / 2)
                y = (y_center * img_height) - (h / 2)

                return int(x), int(y), int(w), int(h)
            else:
                # Handle cases where insufficient values are found in label file
                print(f"Invalid format in label file '{label_path}': Expected at least 4 values for bbox, found {len(bbox_values)}")
                return None
        else:
            # Handle empty label files
            print(f"Empty label file '{label_path}'")
            return None
    except Exception as e:
        # Handle any exceptions during parsing
        print(f"Error parsing label file '{label_path}': {e}")
        return None

# Function to process images according to Process 2
def process_2(image_path, label_path, save_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale (black and white)

    if img is not None and img.size > 0:
        img_height, img_width = img.shape  # Updated line to get height and width (no channels for grayscale)

        bbox = extract_bbox_from_yolov8(label_path, img_width, img_height)

        if bbox is not None:
            x, y, w, h = bbox
            cropped_img = img[max(0, y):min(img_height, y + h), max(0, x):min(img_width, x + w)]

            # Check if the cropped image is valid and has the correct shape
            if cropped_img.size > 0:
                resized_img = cv2.resize(cropped_img, (28, 28))

                # Save the processed image
                cv2.imwrite(save_path, resized_img)

                # Create a text file (with the same name as the image) and write "drone" to it
                text_file_name = os.path.splitext(os.path.basename(save_path))[0] + '.txt'
                text_file_path = os.path.join(os.path.dirname(save_path), text_file_name)
                with open(text_file_path, 'w') as file:
                    file.write("drone")
            else:
                print(f"Skipping {image_path} due to invalid crop or shape.")
        else:
            print(f"Skipping {image_path} due to label parsing error.")
    else:
        print(f"Skipping {image_path} as it's not a valid image.")

# Define paths for the original images and the destination for processed images
og_folder_path = '/content/sample_data/Road2_drone_data/OG'
true_drones_path = '/content/sample_data/Road2_drone_data/True_drones'

# Get a list of image files in the original folder
og_image_files = [file for file in os.listdir(og_folder_path) if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

# Process all images from OG folder using Process 2
if len(og_image_files) > 0:
    for image_name in og_image_files:
        image_path = os.path.join(og_folder_path, image_name)

        label_file_name = os.path.splitext(image_name)[0] + '.txt'
        label_file_path = os.path.join(og_folder_path, label_file_name)

        processed_image_path = os.path.join(true_drones_path, image_name)

        # Process the image according to Process 2
        process_2(image_path, label_file_path, processed_image_path)
else:
    print("No image files found in the 'OG' folder.")


Converting to TensorFlow datasets

In [24]:
import tensorflow as tf
import os

# Assuming labels are strings ('not a drone', 'drone') in the label files
label_to_index = {'not a drone': 0, 'drone': 1}

# Function to load images and labels from directories with categorical encoding
def load_images_and_labels(directory):
    image_data = []
    label_data = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            image_path = os.path.join(directory, filename)

            # Load and preprocess image
            image = tf.io.read_file(image_path)
            image = tf.image.decode_jpeg(image)

            # Append image and encoded label to lists
            image_data.append(image)
            if 'Fake' in directory:
              label_data.append(0)
            else:
              label_data.append(1)

    return image_data, label_data

# Define paths for Fake_ones and True_drones
fake_ones_path = '/content/sample_data/Road2_drone_data/Fake_ones'
true_drones_path = '/content/sample_data/Road2_drone_data/True_drones'

# Load images and labels for Fake_ones and True_drones
fake_ones_images, fake_ones_labels = load_images_and_labels(fake_ones_path)
true_drones_images, true_drones_labels = load_images_and_labels(true_drones_path)

# Convert lists to TensorFlow datasets
fake_ones_dataset = tf.data.Dataset.from_tensor_slices((fake_ones_images, fake_ones_labels))
true_drones_dataset = tf.data.Dataset.from_tensor_slices((true_drones_images, true_drones_labels))

# # Shuffle and batch the datasets (adjust batch sizes as needed)
batch_size = 32  # Adjust batch size as per requirement
fake_ones_dataset = fake_ones_dataset.shuffle(len(fake_ones_images)).batch(batch_size)
true_drones_dataset = true_drones_dataset.shuffle(len(true_drones_images)).batch(batch_size)

# Optionally, perform other transformations like normalization, resizing, etc.
# Example:
# fake_ones_dataset = fake_ones_dataset.map(normalize_images)
# true_drones_dataset = true_drones_dataset.map(normalize_images)

# Print the dataset shapes and types
print("Fake ones dataset:", fake_ones_dataset.element_spec)
print("True drones dataset:", true_drones_dataset.element_spec)


Fake ones dataset: (TensorSpec(shape=(None, 28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
True drones dataset: (TensorSpec(shape=(None, 28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [14]:
import numpy as np
np.array(fake_ones_images).shape

(4070, 28, 28, 1)

Merging

In [25]:
import tensorflow as tf

# Combine fake ones and true drones datasets into a single dataset
combined_dataset = tf.data.Dataset.concatenate(fake_ones_dataset, true_drones_dataset)

# Shuffle the combined dataset
combined_dataset_shuffled = combined_dataset.shuffle(buffer_size=len(combined_dataset), reshuffle_each_iteration=False)

# Calculate the sizes for train and test sets
total_size = len(combined_dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

# Split the dataset into train and test sets
train_dataset = combined_dataset_shuffled.take(train_size)
test_dataset = combined_dataset_shuffled.skip(train_size)

# Print the sizes of train and test sets
print("Train dataset size:", sum(1 for _ in train_dataset))
print("Test dataset size:", sum(1 for _ in test_dataset))

Train dataset size: 204
Test dataset size: 52


Training the model

In [29]:
import matplotlib.pyplot as plt
import numpy as np

# # Normalize pixel values between 0 and 1
# train_images = train_images / 255.0
# test_images = test_images / 255.0

# Define and compile the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(train_dataset, validation_data = test_dataset, epochs=30)



Epoch 1/20
204/204 [==============================] - 2s 6ms/step - loss: 50.2875 - accuracy: 0.6904 - val_loss: 9.5840 - val_accuracy: 0.8348
Epoch 2/20
204/204 [==============================] - 1s 6ms/step - loss: 28.6954 - accuracy: 0.7451 - val_loss: 39.3567 - val_accuracy: 0.4791
Epoch 3/20
204/204 [==============================] - 1s 6ms/step - loss: 27.7845 - accuracy: 0.7617 - val_loss: 8.3545 - val_accuracy: 0.8993
Epoch 4/20
204/204 [==============================] - 1s 6ms/step - loss: 22.3790 - accuracy: 0.7591 - val_loss: 7.3899 - val_accuracy: 0.8956
Epoch 5/20
204/204 [==============================] - 1s 6ms/step - loss: 14.3326 - accuracy: 0.8099 - val_loss: 6.2030 - val_accuracy: 0.9085
Epoch 6/20
204/204 [==============================] - 2s 8ms/step - loss: 18.7743 - accuracy: 0.7740 - val_loss: 12.1227 - val_accuracy: 0.4994
Epoch 7/20
204/204 [==============================] - 2s 11ms/step - loss: 15.7033 - accuracy: 0.7801 - val_loss: 5.7173 - val_accuracy: 0.8

Another way to split data: (we'll need a different way to normalize though)

In [28]:
data_dir = '/content/sample_data/Road2_drone_data'
split_p = 0.2

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=split_p,
  subset="training",
  seed=123,
  image_size=(28, 28),
  batch_size=32)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=split_p,
  subset="validation",
  seed=123,
  image_size=(28, 28),
  batch_size=32)

Found 8140 files belonging to 2 classes.
Using 6512 files for training.
Found 8140 files belonging to 2 classes.
Using 1628 files for validation.
